# Project

Welcome to the group project! The project is based on the [ACM RecSys 2021 Challenge](https://recsys-twitter.com/).

- Detailed information about the task, submission and grading can be found in a [dedicates site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1217340).
- Information about the dataset structure [on this site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1218810).

In [1]:
team_name = "team_5" # your team name e.g. 'team_1'
team_members = [("Simone Andreetto","01635069"),
                ("Adrian Bracher",""),
                ("Dominik Mailer","01634043"),
                ("Andreas Merckel",""),
                ("Dominik Pülke","12019262"),
                ("Sebastian Scholz","01526884"),
                ("Felix Winterleitner","01612776"),
                ("Ahmadou Wagne","12002293")] # [("Jane Doe","012345678"), ("John Doe","012345678")]


In [2]:
print(team_name)
print(team_members)

team_5
[('Simone Andreetto', '01635069'), ('Adrian Bracher', ''), ('Dominik Mailer', '01634043'), ('Andreas Merckel', ''), ('Dominik Pülke', '12019262'), ('Sebastian Scholz', '01526884'), ('Felix Winterleitner', '01612776'), ('Ahmadou Wagne', '12002293')]


In [3]:
path_to_data = '../shared/data/project/training/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week

In [4]:
import os
import re
import csv
import datetime
import model_nn
import pandas as pd


from dataprep import import_data

In [5]:


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    return tweet_id, user_id, input_feats, tweet_timestamp


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                #df = pd.read_csv(f, sep="\x01")
                #print(f"df: {df}")
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row)   
                    reply_pred = reply_pred_model(features) # reply_model
                    #retweet_pred = retweet_pred_model(features) # retweet_model
                    #quote_pred = quote_pred_model(features) # pred_model
                    #fav_pred = fav_pred_model(features) # fav_model
                    
                    print(features)
                    #reply_pred = model_nn.reply_pred_model(features)
                    #retweet_pred = model_nn.retweet_pred_model(features)
                    #quote_pred = model_nn.quote_pred_model(features)
                    #fav_pred = model_nn.fav_pred_model(features)
                    
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')


In [6]:
evaluate_test_set()

NameError: name 'reply_pred_model' is not defined

# NN-Model

In [7]:
import os
import re
import csv
import datetime
import model_nn
import pandas as pd

from dataprep import import_data
import dataprep
from sklearn.preprocessing import StandardScaler

In [8]:
used_features = [
            # "text_tokens",
            "hashtags",
            # "tweet_id",
            "present_media",
            "present_links",
            "present_domains",
            "tweet_type",
            # "language",
            # "tweet_timestamp",
            # "engaged_with_user_id",
            # "engaged_with_user_follower_count",
            "engaged_with_user_following_count",
            "engaged_with_user_is_verified",
            "engaged_with_user_account_creation",
            # "engaging_user_id",
            # "enaging_user_follower_count",
            # "enaging_user_following_count",
            # "enaging_user_is_verified",
            # "enaging_user_account_creation",
            "engagee_follows_engager"
    ]

all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
        
    features = line #.split("\x01")
    
    hashtags = features[all_features_to_idx['hashtags']]
    present_media = features[all_features_to_idx['present_media']]
    present_links = features[all_features_to_idx['present_links']]
    present_domains = features[all_features_to_idx['present_domains']]
    tweet_type = features[all_features_to_idx['tweet_type']]
    engaged_with_user_following_count = features[all_features_to_idx['engaged_with_user_following_count']]
    engaged_with_user_is_verified = features[all_features_to_idx['engaged_with_user_is_verified']]
    engaged_with_user_account_creation = features[all_features_to_idx['engaged_with_user_account_creation']]
    engagee_follows_engager = features[all_features_to_idx['engagee_follows_engager']]
    
    return (hashtags, present_media, present_links, present_domains, tweet_type, engaged_with_user_following_count, engaged_with_user_is_verified, engaged_with_user_account_creation, engagee_follows_engager)


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                df = pd.DataFrame(columns=all_features)
                i = 0
                for row in linereader:
                    df.loc[i] = row[:20]
                    i += 1
                    if i > 10000:
                        break
                df_complete = df.copy()
                df = df.loc[:, used_features]
                df = dataprep.transform_data(df)
                scale = StandardScaler()
                #df = scale.fit_transform(df)
                #df = pd.DataFrame(columns=all_features, data=df)
                df = pd.DataFrame(scale.fit_transform(df.values), columns=df.columns, index=df.index)
                for index, row in df.iterrows():
                    tweet_id = df_complete.iloc[[index]]["tweet_id"]
                    user_id = df_complete.iloc[[index]]["engaging_user_id"]
                    #feature_tuple = parse_input_line(row)   
                    #reply_pred = reply_pred_model(features) # reply_model
                    #retweet_pred = retweet_pred_model(features) # retweet_model
                    #quote_pred = quote_pred_model(features) # pred_model
                    #fav_pred = fav_pred_model(features) # fav_model
                    
                    #print(feature_tuple)
                    #print(df.iloc[[index]]["engaged_with_user_following_count"])
                    reply_pred = model_nn.reply_pred_model(df.iloc[[index]])
                    retweet_pred = model_nn.retweet_pred_model(df.iloc[[index]])
                    quote_pred = model_nn.quote_pred_model(df.iloc[[index]])
                    fav_pred = model_nn.fav_pred_model(df.iloc[[index]])
                    
                    
                    # print(str(tweet_timestamp))
                    print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')


In [ ]:
evaluate_test_set()

0   -0.282289
Name: engaged_with_user_following_count, dtype: float64
[0.07350903] [0.] [0.] [0.43113713]
1   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.41119139] [0.24739607] [0.16010194] [0.]
2   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.41119139] [0.24739607] [0.16010194] [0.]
3   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.41119139] [0.24739607] [0.16010194] [0.]
4   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.41119139] [0.24739607] [0.16010194] [0.]
5   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.41119139] [0.24739607] [0.16010194] [0.]
6   -0.343981
Name: engaged_with_user_following_count, dtype: float64
[0.09457317] [0.07191359] [0.05773246] [0.]
7   -0.34964
Name: engaged_with_user_following_count, dtype: float64
[0.42138164] [0.] [0.] [0.41507649]
8   -0.353629
Name: engaged_with_user_following_count, dtype: float64
[0.44435404] [0.] [0.] [1.]
9   -0.

[0.] [0.] [0.] [1.]
84   -0.35567
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [1.]
85   -0.354519
Name: engaged_with_user_following_count, dtype: float64
[0.48802579] [0.] [0.] [1.]
86   -0.350011
Name: engaged_with_user_following_count, dtype: float64
[0.27308298] [0.55500223] [0.5559581] [0.79818375]
87   -0.350011
Name: engaged_with_user_following_count, dtype: float64
[0.27308298] [0.55500223] [0.5559581] [0.79818375]
88   -0.350011
Name: engaged_with_user_following_count, dtype: float64
[0.27308298] [0.55500223] [0.5559581] [0.79818375]
89   -0.349955
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [1.]
90   -0.348174
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
91   -0.348174
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
92   -0.348174
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
93   -0.348174
Name: engaged_with_user_following_count, dtype

[0.] [0.] [0.] [0.]
171   -0.355057
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
172   -0.355057
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
173   -0.355057
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
174   -0.355057
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
175   -0.355057
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [0.]
176   -0.350957
Name: engaged_with_user_following_count, dtype: float64
[0.91421213] [0.21613153] [0.46219246] [0.72247279]
177   -0.350957
Name: engaged_with_user_following_count, dtype: float64
[0.91421213] [0.21613153] [0.46219246] [0.72247279]
178   -0.350957
Name: engaged_with_user_following_count, dtype: float64
[0.91421213] [0.21613153] [0.46219246] [0.72247279]
179   -0.350957
Name: engaged_with_user_following_count, dtype: float64
[0.91421213] [0.21613153] [0.46219246] [0.72247279]
180   -0.350957
Name: en

[0.] [0.] [0.] [0.]
257   -0.356412
Name: engaged_with_user_following_count, dtype: float64
[0.35907275] [0.] [0.] [0.]
258   -0.353796
Name: engaged_with_user_following_count, dtype: float64
[0.52518388] [0.] [0.71538504] [1.]
259   -0.353796
Name: engaged_with_user_following_count, dtype: float64
[0.52518388] [0.] [0.71538504] [1.]
260   -0.353313
Name: engaged_with_user_following_count, dtype: float64
[0.28752585] [0.] [0.] [0.11379042]
261   -0.353313
Name: engaged_with_user_following_count, dtype: float64
[0.28752585] [0.] [0.] [0.11379042]
262   -0.353313
Name: engaged_with_user_following_count, dtype: float64
[0.28752585] [0.] [0.] [0.11379042]
263   -0.353276
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0238237] [0.]
264   -0.353276
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0238237] [0.]
265   -0.353276
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0238237] [0.]
266   -0.353276
Name: engaged_with_user_fol

[0.70582003] [0.1610174] [0.45211402] [0.]
342   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
343   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
344   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
345   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
346   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
347   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
348   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
349   -0.333553
Name: engaged_with_user_following_count, dtype: float64
[0.70582003] [0.1610174] [0.45211402] [0.]
350   -0.333553
Name: engaged_with_us

...

[0.60141956] [0.] [0.] [0.]
5919   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5920   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5921   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5922   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5923   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5924   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5925   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5926   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5927   -0.356542
Name: engaged_with_user_following_count, dtype: float64
[0.60141956] [0.] [0.] [0.]
5928   -0.356542
Name: engaged_with_user_following_count, dtype

[0.11253262] [0.] [0.] [0.30705018]
6005   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6006   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6007   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6008   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6009   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6010   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6011   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6012   -0.347747
Name: engaged_with_user_following_count, dtype: float64
[0.42745014] [0.4762272] [0.67116274] [0.]
6013   -0.347747
Name: engaged_with_

[0.] [0.] [0.] [0.77665465]
6089   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6090   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6091   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6092   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6093   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6094   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6095   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6096   -0.354334
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.0141081] [0.]
6097   -0.33398
Name: engaged_with_user_following_count, dtype: float64
[0.56622354] [0.] [0.44536209] [0.]
6098   -0.33398
Name: engaged_with_user_following_count, dtype: 

[0.] [0.] [0.] [1.]
6174   -0.334221
Name: engaged_with_user_following_count, dtype: float64
[0.40498601] [0.] [0.] [1.]
6175   -0.334221
Name: engaged_with_user_following_count, dtype: float64
[0.40498601] [0.] [0.] [1.]
6176   -0.351309
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [1.]
6177   -0.316373
Name: engaged_with_user_following_count, dtype: float64
[0.34678665] [0.] [0.] [0.]
6178   -0.316373
Name: engaged_with_user_following_count, dtype: float64
[0.57565747] [0.] [0.] [0.]
6179   -0.354816
Name: engaged_with_user_following_count, dtype: float64
[0.3990098] [0.] [0.] [0.]
6180   -0.323312
Name: engaged_with_user_following_count, dtype: float64
[0.44106398] [0.] [0.] [0.17446807]
6181   -0.299359
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [1.]
6182   -0.299359
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] [1.]
6183   -0.352627
Name: engaged_with_user_following_count, dtype: float64
[0.] [0.] [0.] 

[0.11804153] [0.] [0.] [0.]
6260    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.11804153] [0.] [0.] [0.]
6261    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6262    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6263    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6264    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6265    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6266    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6267    1.805606
Name: engaged_with_user_following_count, dtype: float64
[0.44188005] [0.15483859] [0.30409634] [0.]
6268    1.805606
Name: engaged_with_user_following_c

# End NN-Model

# User-To-User Model

In [ ]:
# import Model class
%run -i "u2u.py"

In [ ]:
#train models
#imported_data = get_imported_data()

#u2u_reply = Predictor("reply", imported_data, dump_to_file=True)
#u2u_retweet = Predictor("retweet", imported_data, dump_to_file=True)
#u2u_quote = Predictor("retweet_with_comment", imported_data, dump_to_file=True)
#u2u_fav = Predictor("like", imported_data, dump_to_file=True)

In [ ]:
# load model from stored files
u2u_reply = load("model_content/u2u_reply.joblib")
u2u_retweet = load("model_content/u2u_retweet.joblib")
u2u_quote = load("model_content/u2u_retweet_with_comment.joblib")
u2u_fav = load("model_content/u2u_like.joblib")

In [ ]:
# Evaluator class that computes the f1 score. Use in evaluate_test_set!
class Evaluator:
    def __init__(self):
        self.tp = 0
        self.tn = 0
        self.fp = 0
        self.fn = 0
        
    def add(self, interaction, prediction):
        true = 0 if interaction == "" else 1
        if true == prediction and true == 0:
            self.tn += 1
        elif true == prediction and true == 1:
            self.tp += 1
        elif true != prediction and true == 0:
            self.fp += 1
        elif true != prediction and true == 1:
            self.fn += 1
            
    def get_f1(self):
        try:
            return self.tp / (self.tp + 0.5 * (self.fn + self.fp))
        except:
            return -1

In [ ]:
# User-To-User Model

path_to_data = '../../shared/data/project/validation/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "retweet", "reply", "like", "retweet_with_comment"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))


# adapted to also return the true interaction values
def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    retweet = features[all_features_to_idx['retweet']]
    reply = features[all_features_to_idx['reply']]
    like = features[all_features_to_idx['like']]
    retweet_with_comment = features[all_features_to_idx['retweet_with_comment']]

    return tweet_id, user_id, input_feats, tweet_timestamp, retweet, reply, like, retweet_with_comment


#adapted to return the f1 values for each interaction category
def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
    
    reply_eval = Evaluator()
    retweet_eval = Evaluator()
    quote_eval = Evaluator()
    fav_eval = Evaluator()
    
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp, retweet, reply, fav, quote = parse_input_line(row)   
                    reply_pred = u2u_reply.predict(user_id, tweet_id, binary=True) # reply_model
                    retweet_pred = u2u_retweet.predict(user_id, tweet_id, binary=True) # retweet_model
                    quote_pred = u2u_quote.predict(user_id, tweet_id, binary=True) # pred_model
                    fav_pred = u2u_fav.predict(user_id, tweet_id, binary=True) # fav_model
                    
                    reply_eval.add(reply, reply_pred)
                    retweet_eval.add(retweet, retweet_pred)
                    quote_eval.add(quote, quote_pred)
                    fav_eval.add(fav, fav_pred)
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')
        return reply_eval.get_f1(), retweet_eval.get_f1(), quote_eval.get_f1(), fav_eval.get_f1()




In [ ]:
reply_eval, retweet_eval, quote_eval, fav_eval = evaluate_test_set()

print(f"F1-Reply: {reply_eval}")
print(f"F1-Retweet: {reply_tweet}")
print(f"F1-Quote: {reply_quote}")
print(f"F1-Like: {reply_fav}")

# -- End User-To-User Model --

In [ ]:

data = import_data(path_to_data+dataset_type)
data

In [ ]:
print(" -- unique tweets --")
print(data["tweet_id"].unique().size)
print(" -- engaged users --")
print(data["engaged_with_user_id"].unique().size)
print(" -- engaging users --")
print(data["engaging_user_id"].unique().size)

In [ ]:
# hidden


# Metrics Functions



## Recsys Challenge Functions

In [ ]:
from sklearn.metrics import average_precision_score, log_loss

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

  
ground_truth = read_predictions(path_to_data + dataset_type) # will return data in the form (tweet_id, user_id, labed (1 or 0))
predictions = read_predictions("predictions.csv") # will return data in the form (tweet_id, user_id, prediction)

rce = compute_rce(predictions, ground_truth)
average_precision = average_precision_score(ground_truth, predictions)

print(rce)
print(average_precision)

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit